In [2]:
import numpy as np
np.set_printoptions(precision=5) # 2 decimals when printing arrays
np.set_printoptions(suppress=True) # No scientific notation for small numbers
import matplotlib.pyplot as plt

from quantum_systems import ODQD, GeneralOrbitalSystem

from coupled_cluster.ccd.rhs_t import compute_t_2_amplitudes
from coupled_cluster.ccsd.rhs_t import compute_t_1_amplitudes
import coupled_cluster.ccsd.rhs_t as sdrhs
from coupled_cluster.mix import AlphaMixer, DIIS
from coupled_cluster.ccd import CCD
from coupled_cluster.ccsd import CCSD

ModuleNotFoundError: No module named 'quantum_systems'

In [2]:
def pprint(tensor):
    print(tensor[:, :, 1, 0].real)

## Setting up the system

In [83]:
l_0 = 10                 # number of basis functions
grid_length = 10        # compute from x = -10 to x = 10 in 1D
num_grid_points = 2001
omega = 0.25            # strength of harmonic oscillator potential
n = 4                   # number of particles

odho = ODQD(l_0, grid_length, num_grid_points, a = 0.25, alpha = 1, potential = ODQD.HOPotential(omega))
system = GeneralOrbitalSystem(n = n, basis_set=odho)

### CCSD

In [84]:
ccsd = CCSD(system, mixer = DIIS, verbose=False)
ccsd.setup_t_mixer(np=np)
ccsd.compute_ground_state()
print(f"Ground state energy: {ccsd.compute_energy().real:.6}")

Ground state energy: 0.541379


In [85]:
ccsd.compute_ground_state()
print(f"Ground state energy: {ccsd.compute_energy():.6}")

Ground state energy: (0.825315+0j)


In [24]:
pprint(ccsd.t_2)

[[-0.       0.6855  -0.      -0.      -0.       0.23397 -0.      -0.     ]
 [-0.6855  -0.       0.      -0.      -0.23397 -0.       0.      -0.     ]
 [-0.      -0.      -0.       0.30334 -0.       0.      -0.       0.13369]
 [ 0.      -0.      -0.30334 -0.      -0.      -0.      -0.13369 -0.     ]
 [-0.       0.23397 -0.       0.      -0.       0.15818 -0.      -0.     ]
 [-0.23397 -0.      -0.      -0.      -0.15818 -0.       0.      -0.     ]
 [-0.      -0.      -0.       0.13369 -0.      -0.      -0.       0.09357]
 [ 0.      -0.      -0.13369 -0.       0.      -0.      -0.09357 -0.     ]]


In [56]:
v,o = ccsd.v, ccsd.o
f = ccsd.f
t_1 = ccsd.t_1
u = ccsd.u
t_2 = ccsd.t_2

In [63]:
out = np.copy(f[v, o])
out += np.tensordot(f[v, v], t_1, axes=((1), (0)))
out += -np.tensordot(f[o, o], t_1, axes=((0), (1))).transpose(1, 0)
out += np.tensordot(u[v, o, o, v], t_1, axes=((1, 3), (1, 0)))
W_ki = -np.dot(f[o, v], t_1)
out += np.dot(t_1, W_ki)
W_akdi = np.tensordot(u[v, o, v, v], t_1, axes=((2), (0)))
out += np.tensordot(W_akdi, t_1, axes=((1, 2), (1, 0)))
W_ki = -np.tensordot(u[o, o, o, v], t_1, axes=((1, 3), (1, 0)))
out += np.dot(t_1, W_ki)
W_kc = -np.tensordot(u[o, o, v, v], t_1, axes=((1, 3), (1, 0)))
W_ki = np.dot(W_kc, t_1)
out += np.dot(t_1, W_ki)
out += np.tensordot(f[o, v], t_2, axes=((0, 1), (3, 1)))
out += 0.5 * np.tensordot(u[v, o, v, v], t_2, axes=((1, 2, 3), (3, 0, 1)))
term = np.tensordot(u[o, o, o, v], t_2, axes=((0, 1), (2, 3)))
out -= 0.5 * np.trace(term, axis1=1, axis2=3).swapaxes(0, 1)
W_kldi = -0.5 * np.tensordot(u[o, o, v, v], t_1, axes=((2), (0)))
out += np.tensordot(t_2, W_kldi, axes=((1, 2, 3), (2, 0, 1)))
W_ki = -0.5 * np.tensordot(u[o, o, v, v], t_2, axes=((1, 2, 3), (3, 0, 1)))
out += np.dot(t_1, W_ki)
temp_ld = np.tensordot(u[o, o, v, v], t_1, axes=((0, 2), (1, 0)))
out += np.tensordot(temp_ld, t_2, axes=((0, 1), (2, 0)))
out.real

array([[-0.     ,  0.     ],
       [ 0.     , -0.     ],
       [-0.35986,  0.     ],
       [ 0.     , -0.35986],
       [-0.     ,  0.     ],
       [ 0.     , -0.     ],
       [-0.17758,  0.     ],
       [ 0.     , -0.17758]])

In [22]:
ccsd.rhs_t_1.fill(0)
(sdrhs.compute_t_1_amplitudes(ccsd.f, ccsd.u, ccsd.t_1, ccsd.t_2, ccsd.o, ccsd.v, out=ccsd.rhs_t_1, np=np)).real # / ccsd.d_t_1

array([[-0.     ,  0.     ],
       [ 0.     , -0.     ],
       [-0.35986,  0.     ],
       [ 0.     , -0.35986],
       [-0.     ,  0.     ],
       [ 0.     , -0.     ],
       [-0.17758,  0.     ],
       [ 0.     , -0.17758]])

In [ ]:
ccsd.rhs_t_2.fill(0)
compute_t_2_amplitudes(ccsd.f, ccsd.u, ccsd.t_2, ccsd.o, ccsd.v, out = ccsd.rhs_t_2, np = np)
direction_vector_1 = np.divide(ccsd.rhs_t_2, ccsd.d_t_2)
trial_vector_1 = np.copy(ccsd.t_2)
error_vector_1 = np.copy(ccsd.rhs_t_2)

ccd.t_2 = ccd.t_2_mixer.compute_new_vector(trial_vector_1, direction_vector_1, error_vector_1)

In [ ]:
pprint(trial_vector_1)
pprint(direction_vector_1)
pprint(error_vector_1)

In [ ]:
pprint(ccsd.t_2)

In [69]:

ccsd.compute_ground_state()
print(f"Ground state energy: {ccsd.compute_energy():.6}")

AssertionError: The t amplitudes did not converge. Last residual: [0.33145401375307415, 2.931746734004814]